In [3]:
import json
import pandas as pd
import gf_utils
from gf_utils.stc_data import get_stc_data
from pathlib import Path
from datetime import datetime
from collections import defaultdict

log_dir = Path('logs/equip')
output_file = Path('equip.csv')
full_record_df = pd.read_csv(output_file)
game_data = get_stc_data(Path(gf_utils.__file__).resolve().parent/'../../data/ch/formatted/json')

In [19]:
from rich.table import Table
from rich.console import Console
from rich.style import NULL_STYLE
console = Console(force_terminal=True,force_jupyter=False,no_color=True,highlight=False)
for formula in game_data['recommended_formula'].values():
    # print(formula)
    if formula['develop_type']!=3:
        continue
    mp,ammo,mre,part = formula['mp'],formula['ammo'],formula['mre'],formula['part']
    common_dev_df = full_record_df.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and item_num==1 and trust==True')['equip_id'].value_counts().to_frame('count')
    common_dev_df['chance'] = common_dev_df['count'].apply(lambda x: round(x/common_dev_df['count'].sum()*100,3))
    special_dev_df = full_record_df.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and item_num==0 and trust==True')['equip_id'].value_counts().to_frame('count')
    special_dev_df['chance'] = special_dev_df['count'].apply(lambda x: round(x/special_dev_df['count'].sum()*100,3))
    dev_df = common_dev_df.join(special_dev_df,rsuffix='_special',how='outer').fillna(0).reset_index()
    dev_df['count_special'] = dev_df['count_special'].astype(int)
    for key in ['name','rank']:
        dev_df[key] = dev_df['index'].apply(lambda x: game_data['equip'][x][key])
    footer = f"Count: common {dev_df['count'].sum()}, special {dev_df['count_special'].sum()}\n"
    dev_df = dev_df.query('rank==5').sort_values('chance',ascending=False)
    footer += f"Rank5: common {dev_df['chance'].sum():.3f}, special {dev_df['chance_special'].sum():.3f}"
    table = Table('           name','common','special',header_style=NULL_STYLE, title_style=NULL_STYLE, style=NULL_STYLE, caption=footer, caption_justify='left', caption_style=NULL_STYLE)
    for col in table.columns:
        col.justify='right'
    table.title = f"Formula: {mp} {ammo} {mre} {part}"
    for idx, record in dev_df.iterrows():
        table.add_row(record['name'],*[f'{record[k]:.3f}' for k in ['chance','chance_special']])
    console.print(table)
    console.print()

        Formula: 140 10 110 10        
┌─────────────────┬────────┬─────────┐
│            name │ common │ special │
├─────────────────┼────────┼─────────┤
│         EOT 518 │  1.319 │   0.803 │
│        ITI MARS │  0.694 │   0.357 │
│     VFL 6-24X56 │  0.556 │   2.141 │
│       AC4消音器 │  0.069 │   0.000 │
│      RMR T4红点 │  0.069 │   0.000 │
└─────────────────┴────────┴─────────┘
Count: common 1440, special 1121      
Rank5: common 2.707, special 3.301    

        Formula: 140 10 110 10        
┌─────────────────┬────────┬─────────┐
│            name │ common │ special │
├─────────────────┼────────┼─────────┤
│         EOT 518 │  1.319 │   0.803 │
│        ITI MARS │  0.694 │   0.357 │
│     VFL 6-24X56 │  0.556 │   2.141 │
│       AC4消音器 │  0.069 │   0.000 │
│      RMR T4红点 │  0.069 │   0.000 │
└─────────────────┴────────┴─────────┘
Count: common 1440, special 1121      
Rank5: common 2.707, special 3.301    

        Formula: 140 10 110 10        
┌─────────────────┬────────┬─────

In [77]:
full_record_df.query('mp==140 and ammo==10 and mre==110 and part==10')

,id,user_id,dev_type,build_slot,dev_uname,dev_lv,equip_id,mp,ammo,mre,part,input_level,core,item_num,dev_time,fairy_id,passive_skill,quality_lv,file,trust
415,28839658,962632,0,1,高处不胜寒,90,1,140,10,110,10,0,0,0,1666931765,0,0,0,221028-125505.json,False
416,28839659,962632,0,3,高处不胜寒,90,1,140,10,110,10,0,0,0,1666931765,0,0,0,221028-125505.json,False
435,28839689,530322,0,1,Goring,160,1,140,10,110,10,0,0,0,1666932485,0,0,0,221028-125505.json,False
436,28839690,530322,0,3,Goring,160,1,140,10,110,10,0,0,0,1666932485,0,0,0,221028-125505.json,False
437,28839691,530322,0,5,Goring,160,2,140,10,110,10,0,0,0,1666932485,0,0,0,221028-125505.json,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73803,28933482,526212,0,3,時間帶走歲月,103,1,140,10,110,10,0,0,0,1668643675,0,0,0,221117-081234.json,True
73806,28933485,665272,0,1,starofsky,55,11,140,10,110,10,0,0,1,1668643894,0,0,0,221117-082327.json,True
73814,28933493,993122,0,1,Lanoom,77,2,140,10,110,10,0,0,0,1668644190,0,0,0,221117-082327.json,True
73815,28933494,993122,0,3,Lanoom,77,2,140,10,110,10,0,0,0,1668644190,0,0,0,221117-082327.json,True


In [18]:
full_record_df.query('user_id==962632')['item_num'].value_counts()

0    53
1    34
Name: item_num, dtype: int64

In [7]:
[f'{record[k]:.3f}' for k in ['name','chance','chance_special']]

['Mk2', '0.031', '0.0']

In [15]:
f'{0.0:.3}'

'0.0'